In [13]:
### Importing ###
from numpy import exp, array, random, dot #imports linear algebra operations/types
import numpy as np #operations in numpy
from scipy import misc #misc. operations in scipy
import skimage
import math 
from __future__ import with_statement

In [14]:
### Get picture data ###
arr1 = np.divide(misc.imread('save_save_save_america02.png'),255) #read 1st image, divide by 255 for number between 0 and 1
arr1r = arr1[:,:,0] #reds
arr1g = arr1[:,:,1] #greens
arr1b = arr1[:,:,2] #blues
arr2 = np.divide(misc.imread('tAmerica2.png'),255)
arr2r = arr2[:,:,0]
arr2g = arr2[:,:,1]
arr2b = arr2[:,:,2]

In [26]:
### Hyper-parameters ###
layerSize = 30 #number of filters in a layer
layerNum = 3 #number of layers
filterSize = 3 #size of filter
epochs = 10 #number of times trained
filters = [] #init filter
bias = 0.01
targets = np.ones((len(arr1r)*len(arr1r[0])*layerSize)) #target values

In [27]:
### Variables based on hyper-params ###
filterNum = layerSize**2 #number of filters per layer
totalFilterNum = filterNum*layerNum #total number of filters

In [28]:
### Set filters ###
def setFilters():
    global filters #Filters is a global variable
    filters = 0.01*random.random((layerNum, filterNum, filterSize, filterSize))-0.005 #Set filters to random 4d array
setFilters()
print(filters)

[[[[ -3.64212952e-03   2.90344116e-03  -3.63817655e-03]
   [  2.32708067e-03   4.69171657e-03   4.33176425e-03]
   [  3.24594836e-03   2.98314503e-03   1.40891572e-03]]

  [[ -2.78612659e-03  -2.12143949e-04   3.07737480e-03]
   [  4.47170856e-03  -5.04215412e-04   3.50104093e-03]
   [ -3.14571840e-03  -2.04947062e-04  -1.36703756e-03]]

  [[ -4.21614588e-03   2.11389774e-03   3.44428978e-03]
   [  8.41532931e-04  -4.47485336e-03   2.07061917e-03]
   [ -4.06369263e-03   3.08984921e-03   4.79958936e-03]]

  ..., 
  [[  4.74215951e-03  -1.06786966e-03  -1.69972541e-03]
   [ -2.70899350e-03   3.34040364e-03  -3.41510849e-04]
   [ -3.19125313e-03  -4.84400266e-03  -3.93309850e-03]]

  [[ -9.97224287e-04  -3.12132106e-03  -4.36613504e-03]
   [ -8.66840510e-04   1.43947922e-03  -6.63015663e-04]
   [  1.12994189e-03   4.97048779e-03   4.11373093e-03]]

  [[ -3.43088792e-03  -1.09938195e-03   3.10065182e-03]
   [  2.28344337e-03   3.87323862e-03   1.33554317e-03]
   [  1.22997064e-05  -2.70354

In [29]:
### Helper functions ###
#Activation function (Leaky ReLU)
def activation(x, deriv=False, leak=-0.01):
    new_x=np.copy(array(x)) #Create a copy of x
    if deriv: #If you want to activate derivative
        #Calculation of derivative of leaky ReLU
        new_x[x > 0] = 1 
        new_x[x <= 0] = leak
        return new_x
    #Calculation of leaky ReLU
    new_x[x < 0] *= leak
    new_x = x * (x > 0)
    return new_x
#Sum with no negatives (not used)
'''
def sumNoNeg(arr):
    result = 0
    for y in range(len(arr)):
        for x in range(len(arr[0])):
            result += max(arr[y][x],0)
    return result
'''
#Apply cross-correlation for 1 index
def apply(filterArr, img, row, col):
    value = 0 #Init value
    halfFil = int(filterSize/2) #Half of filterSize
    padImg = np.pad(img, [(halfFil,halfFil),(halfFil,halfFil)],'constant', constant_values=(0)) #Pad image with 0's
    for x in range(filterSize): 
        for y in range(filterSize): #Apply over all filter
            value += padImg[x+row][y+col]*filterArr[x][y] #Add image multiplied by filter
    return value
#Softmax function
def softmax(x):
#     exps = np.exp(x - x.max())
#     return exps / np.sum(exps)
    print(x)
    return np.exp(x) / np.sum(np.exp(x), axis=0) #Calculate softmax
print(apply(filters[1][1],arr1r,0,0))

0.0


In [30]:
### Apply filters ###
def applyFilters(img, new_img, filterArr):
    for y in range(len(img)):
        for x in range(len(img[y])): #Apply over all image
            new_img[y][x] += activation(apply(filterArr,img,y,x))+bias#Add up all cross-correlations (with ReLU)
    return new_img

In [31]:
### Pooling ###
def pool(img, size):
    pad_w = size-len(img)%size
    pad_h = size-len(img[0])%size
    if pad_w == size:
        pad_w = 0
    if pad_h == size:
        pad_h = 0
    new_img = np.lib.pad(array(img), ((0,pad_w),(0,pad_h)), 'edge') #Pad image
    k = len(new_img)/size #Width of new image 
    l = len(new_img[0])/size #Height of new image
    return new_img.reshape(int(k),size,int(l),size).max(axis=(-1,-3)) #Arrange image into groups of size*size, then get max on axis

In [32]:
### Forward-propagation ###
def forward(img):
    outputLayers = [[img[:,:,0],img[:,:,1],img[:,:,2]]] #Get all the image 
    for num in range(layerNum):
        outputLayers.append([]) #Append a new layer
        for lay in range(layerSize):
            outputLayers[num+1].append(np.zeros((len(outputLayers[num][lay]), len(outputLayers[num][lay][0]))))
            # Add components of the layer
            outputLayers[num][lay] = pool(outputLayers[num][lay], 2)
            # Do pooling
        #lay1 is the layer index you are at and lay2 is the layer index you are going to
        for lay1 in range(layerSize):
            for lay2 in range(layerSize):
                newImage = outputLayers[num+1][lay2] #Get refrence to next layer
#                 newImage += bias
#                 print("S-O: ("+str(len(outputLayers))+", "+str(len(outputLayers[0]))+")")
#                 print("S-F: ("+str(len(filters))+", "+str(len(filters[0]))+")")
                applyFilters(outputLayers[num][lay1], newImage, filters[num][layerSize*lay1+lay2])
                #Apply corresponding filter
                if (filterNum*num + layerSize*lay1+lay2) % 20 == 0:
                    print(filterNum*num + layerSize*lay1+lay2)
#                 print(newImage)
    outputLayers.append(softmax(array(outputLayers[len(outputLayers)-1]).flatten())) #Add flattened softmaxed array
    return outputLayers
out = forward(arr1)
print(out)

IndexError: list index out of range

In [33]:
### Error + Back-propagation ###
def back(layer, grad):
    error = -np.sum(np.multiply(np.log10(out[len(out)-1]),targets)) # negative sum of log(out) times target- cross-entropy
    print("ERROR: "+str(error))
    if layer==0:
        return
    print(layer)
    #keep array of all gradients/changes/deltas
    new_grad = np.zeros((filterNum,filterSize,filterSize))
    for x in range(filterNum):
        r = 0
#        0 print("Filter S:"+str(len(filters)))
#         print(layerNum)
        for row in filters[layer-1][x]:
            c=0
            for w in row:
                new_grad[x][r][c]=0.01*np.multiply(error,np.mean(np.multiply(np.multiply(
                    activation(array(out[layer]),deriv=True),
                    w),
                    grad[x][r][c])))
#                 print(-0.01*np.multiply(error,np.mean(np.multiply(np.multiply(
#                     activation(array(out[layer]),deriv=True),
#                     w),
#                     grad[x][r][c]))))
#                 print(-np.divide(error,np.sum(np.multiply(np.multiply(out[layer-1], w), grad[x][r][c]))))
#                 print(new_grad[x][r][c])
                filters[layer-1][x][r][c]+=new_grad[x][r][c]
                c+=1
            r+=1
    back(layer-1, new_grad)
backout = back(layerNum, np.ones((filterNum,filterSize,filterSize)))
#take all output from prev. layer, multiply by corresponding gradient = new deltas 

ValueError: operands could not be broadcast together with shapes (1200,) (12000,) 

In [34]:
setFilters()
np.seterr(divide='raise')
np.seterr(invalid='raise')
np.seterr(over='raise')

{'divide': 'raise', 'invalid': 'raise', 'over': 'raise', 'under': 'ignore'}

In [35]:
for x in range(epochs):
    out = forward(arr1)
#     error = -np.sum(np.multiply(np.log10(out[len(out)-1]),targets))
#     print(error)
#     print(filters)
    back(layerNum, np.ones((filterNum,filterSize,filterSize)))
#     print(filters)

IndexError: list index out of range